In [1]:
import magma as m
import os
os.environ["MANTLE_TARGET"] = "coreir"
import mantle

class FullAdder(m.Circuit):
    name = "FullAdder"
    IO = ["a", m.In(m.Bit), "b", m.In(m.Bit), "cin", m.In(m.Bit),
          "out", m.Out(m.Bit), "cout", m.Out(m.Bit)]
    @classmethod
    def definition(io):
        # Generate the sum
        _sum = mantle.XOr(3)(io.a, io.b, io.cin)
        m.wire(_sum, io.out)
        # Generate the carry
        carry = mantle.Or(3)(
            mantle.And(2)(io.a, io.b), 
            mantle.And(2)(io.b, io.cin), 
            mantle.And(2)(io.a, io.cin)
        )
        m.wire(carry, io.cout)

In [2]:
T = m.Bits(4)
class Adder4(m.Circuit):
    name = "Adder4"
    IO = ["a", m.In(T), "b", m.In(T), "cin", m.In(m.Bit),
          "out", m.Out(T), "cout", m.Out(m.Bit)]
    @classmethod
    def definition(io):
        adder1 = FullAdder()
        m.wire(io.a[0], adder1.a)
        m.wire(io.b[0], adder1.b)
        m.wire(io.cin, adder1.cin)
        adder2 = FullAdder()
        m.wire(io.a[1], adder2.a)
        m.wire(io.b[1], adder2.b)
        m.wire(adder1.cout, adder2.cin)
        adder3 = FullAdder()
        m.wire(io.a[2], adder3.a)
        m.wire(io.b[2], adder3.b)
        m.wire(adder2.cout, adder3.cin)
        adder4 = FullAdder()
        m.wire(io.a[3], adder4.a)
        m.wire(io.b[3], adder4.b)
        m.wire(adder3.cout, adder4.cin)
        
        m.wire(adder4.cout, io.cout)
        m.wire(m.bits([adder1.out, adder2.out, adder3.out, adder4.out]), io.out)

In [3]:
from magma.simulator import PythonSimulator
from magma.bit_vector import BitVector

simulator = PythonSimulator(Adder4)
simulator.set_value(Adder4.a, BitVector(2, num_bits=4))
simulator.set_value(Adder4.b, BitVector(3, num_bits=4))
simulator.set_value(Adder4.cin, True)
simulator.evaluate()
assert simulator.get_value(Adder4.out) == BitVector(6, num_bits=4)
assert simulator.get_value(Adder4.cout) == False
print("Success!")

Success!
